In [2]:
import numpy as np
from scipy import integrate
from matplotlib import pyplot
import time

In [2]:
intervals = [[0.0,1.0],[1.0,2.0]]
(left,right),(bottom,top) = intervals
left, right, bottom, top

(0.0, 1.0, 1.0, 2.0)

In [3]:
def fem2d_linear(Nx, Ny, intervals, mesh_type = "tri"):
    def mesh_tri_p1():
        #unpack intervals
        (l_x, r_x), (l_y, r_y) = intervals 
        hx = (r_x - l_x) / Nx
        hy = (r_y - l_y) / Ny

        P = np.zeros(shape = (2, (Nx + 1) * (Ny + 1)))
        for i in range(Nx + 1):
            for j in range(Ny + 1):
                P[0, i * (Ny + 1) + j] = l_x + i * hx
                P[1, i * (Ny + 1) + j] = l_y + j * hy
        
        T = np.zeros(shape = (3, 2 * Nx * Ny), dtype=int)
        for n in range(2 * Nx * Ny):
            i = n // (2 * Ny)
            j = n % (2 * Ny)
            j1, j2 = j // 2, j % 2
            if j2 == 0:
                T[0, n] = i * (Ny + 1) + j1
                T[1, n] = (i + 1) * (Ny + 1) + j1
                T[2, n] = i * (Ny + 1) + j1 + 1
            else:
                T[0, n] = i * (Ny + 1) + j1 + 1
                T[1, n] = (i + 1) * (Ny + 1) + j1
                T[2, n] = (i + 1) * (Ny + 1) + j1 + 1
        
        boundaryedges = np.zeros(shape = (4, 2 * (Nx + Ny)), dtype=int)
        boundaryedges[0, :] = -1 # all Dirichlet boundaries
        
        st_points = [[0, 0], [Nx - 1, 1], [Nx - 1, 2 * Ny - 1], [0, 2 * Ny - 2]]
        directions = [[1, 0], [0, 2], [-1, 0], [0, -2]]

        N = [Nx, Ny]
        n = 0
        for idx in range(4):
            st_point = st_points[idx]
            direction = directions[idx]
            for k in range(N[int(idx % 2)]):
                i = st_point[0] + k * direction[0]
                j = st_point[1] + k * direction[1]
                element_id = i * 2 * Ny + j
                #print(element_id)
                boundaryedges[1, n] = element_id
                if idx == 0:
                    boundaryedges[2:, n] = T[0:2, element_id]
                elif idx == 1:
                    boundaryedges[2:, n] = T[1:3, element_id]
                elif idx == 2 or idx == 3:
                    boundaryedges[2, n] = T[2, element_id]
                    boundaryedges[3, n] = T[0, element_id]
                
                n += 1

        return P, T, boundaryedges
    
    return mesh_tri_p1()

In [6]:
P, T, boundaryedges = fem2d_linear(20, 100,[[5.0, 6.0],[5.0, 10.0]])
P, T

(array([[ 5.  ,  5.  ,  5.  , ...,  6.  ,  6.  ,  6.  ],
        [ 5.  ,  5.05,  5.1 , ...,  9.9 ,  9.95, 10.  ]]),
 array([[   0,    1,    1, ..., 2018, 2018, 2019],
        [ 101,  101,  102, ..., 2118, 2119, 2119],
        [   1,  102,    2, ..., 2119, 2019, 2120]]))